Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
batch_size = 128
num_steps = 3001
reg = 1e-3

graph = tf.Graph()

with graph.as_default():
    # input data
    # for the training data, use a placeholder that will be fed at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # variables 
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    # training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    + reg * tf.nn.l2_loss(weights)
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # predictions for the training, validation, and test data
    train_pred = tf.nn.softmax(logits)
    valid_pred = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights)+biases)
    test_pred = tf.nn.softmax(tf.matmul(tf_test_dataset, weights)+biases)
    
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_pred.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_pred.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.672709
Minibatch accuracy: 16.4%
Validation accuracy: 22.7%
Minibatch loss at step 500: 2.263530
Minibatch accuracy: 69.5%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 0.955082
Minibatch accuracy: 79.7%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 1.289765
Minibatch accuracy: 78.9%
Validation accuracy: 76.6%
Minibatch loss at step 2000: 1.301163
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 2500: 1.219491
Minibatch accuracy: 77.3%
Validation accuracy: 78.1%
Minibatch loss at step 3000: 0.974949
Minibatch accuracy: 76.6%
Validation accuracy: 77.6%
Test accuracy: 85.6%


In [11]:
batch_size = 128
hidden_size = 1024
num_steps = 3001
reg = 1e-3

graph = tf.Graph()
with graph.as_default():
    # input data
    # for the training data, use a placeholder that will be fed at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # variables for 2-layer
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_size]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    # training computation
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1), W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
    + reg * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # predictions for the training, validation, and test data
    train_pred = tf.nn.softmax(logits)
    valid_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1)+b1), W2)+b2)
    test_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1)+b1),W2)+b2)

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomly pick an offset with training dataset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # prepare a dictionary to feed the minibatch
        # dictionary_key: the placeholder node of the graph to be fed
        # dictionary_value: the numpy array to feed to it
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_pred.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_pred.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 324.939697
Minibatch accuracy: 15.6%
Validation accuracy: 36.2%
Minibatch loss at step 500: 15.244200
Minibatch accuracy: 76.6%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 9.550577
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 7.445848
Minibatch accuracy: 78.9%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 4.565786
Minibatch accuracy: 84.4%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 4.461185
Minibatch accuracy: 87.5%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 5.027860
Minibatch accuracy: 79.7%
Validation accuracy: 82.3%
Test accuracy: 89.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
batch_size = 128
hidden_size = 1024
num_steps = 3001
reg = 1e-3

n_batches = 5
limited_train_dataset = train_dataset[0: batch_size*n_batches]
limited_train_labels = train_labels[0: batch_size*n_batches]

graph = tf.Graph()
with graph.as_default():
    # input data
    # for the training data, use a placeholder that will be fed at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # variables for 2-layer
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_size]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    # training computation
    logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1), W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
    + reg * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # predictions for the training, validation, and test data
    train_pred = tf.nn.softmax(logits)
    valid_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1)+b1), W2)+b2)
    test_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1)+b1),W2)+b2)

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomly pick an offset with training dataset
        offset = (step * batch_size) % (limited_train_labels.shape[0] - batch_size)
        # generate a minibatch
        batch_data = limited_train_dataset[offset:(offset + batch_size), :]
        batch_labels = limited_train_labels[offset:(offset + batch_size), :]
        # prepare a dictionary to feed the minibatch
        # dictionary_key: the placeholder node of the graph to be fed
        # dictionary_value: the numpy array to feed to it
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_pred.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_pred.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 354.422058
Minibatch accuracy: 14.1%
Validation accuracy: 27.4%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Test accuracy: 82.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [14]:
batch_size = 128
hidden_size = 1024
num_steps = 3001
reg = 1e-3
dropout = 0.5

n_batches = 5
limited_train_dataset = train_dataset[0: batch_size*n_batches]
limited_train_labels = train_labels[0: batch_size*n_batches]

graph = tf.Graph()
with graph.as_default():
    # input data
    # for the training data, use a placeholder that will be fed at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # variables for 2-layer
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_size]))
    b1 = tf.Variable(tf.zeros([hidden_size]))
    W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    # training computation
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
    logits = tf.matmul(tf.nn.dropout(h1, dropout), W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
    + reg * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2))
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # predictions for the training, validation, and test data
    train_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1)+b1)*dropout, W2)+b2)
    valid_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1)+b1)*dropout, W2)+b2)
    test_pred = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1)+b1)*dropout,W2)+b2)

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomly pick an offset with training dataset
        offset = (step * batch_size) % (limited_train_labels.shape[0] - batch_size)
        # generate a minibatch
        batch_data = limited_train_dataset[offset:(offset + batch_size), :]
        batch_labels = limited_train_labels[offset:(offset + batch_size), :]
        # prepare a dictionary to feed the minibatch
        # dictionary_key: the placeholder node of the graph to be fed
        # dictionary_value: the numpy array to feed to it
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_pred.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_pred.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 461.882599
Minibatch accuracy: 10.9%
Validation accuracy: 27.4%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1000: 0.945747
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Test accuracy: 85.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [24]:
batch_size = 128
hidden1_size = 1024
hidden2_size = 512
hidden3_size = 256
num_steps = 20001
dropout = 0.5

graph = tf.Graph()
with graph.as_default():
    # input data
    # for the training data, use a placeholder that will be fed at run time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # variables for 4-layer
    W1 = tf.Variable(tf.truncated_normal([image_size*image_size, hidden1_size],
                                        stddev=np.sqrt(2.0/(image_size*image_size))))
    b1 = tf.Variable(tf.zeros([hidden1_size]))
    W2 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size],
                                        stddev=np.sqrt(2.0/hidden1_size)))
    b2 = tf.Variable(tf.zeros([hidden2_size]))
    W3 = tf.Variable(tf.truncated_normal([hidden2_size, hidden3_size],
                                        stddev=np.sqrt(2.0/hidden2_size)))
    b3 = tf.Variable(tf.zeros([hidden3_size]))
    W4 = tf.Variable(tf.truncated_normal([hidden3_size, num_labels],
                                        stddev=np.sqrt(2.0/hidden3_size)))
    b4 = tf.Variable(tf.zeros([num_labels]))
    # training computation
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
    drop1 = tf.nn.dropout(h1, dropout)
    h2 = tf.nn.relu(tf.matmul(drop1, W2) + b2)
    drop2 = tf.nn.dropout(h2, dropout)
    h3 = tf.nn.relu(tf.matmul(drop2, W3) + b3)
    logits = tf.matmul(tf.nn.dropout(h3, dropout), W4) + b4
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
    #learning rate
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.9, staircase=True)
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    # predictions for the training, validation, and test data
    train_pred = tf.nn.softmax(logits)
    h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1)+b1)
    h2_valid = tf.nn.relu(tf.matmul(h1_valid, W2)+b2)
    h3_valid = tf.nn.relu(tf.matmul(h2_valid, W3)+b3)
    valid_pred = tf.nn.softmax(tf.matmul(h3_valid, W4)+b4)
    h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1)+b1)
    h2_test = tf.nn.relu(tf.matmul(h1_test, W2)+b2)
    h3_test = tf.nn.relu(tf.matmul(h2_test, W3)+b3)
    test_pred = tf.nn.softmax(tf.matmul(h3_test, W4)+b4)

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # randomly pick an offset with training dataset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # prepare a dictionary to feed the minibatch
        # dictionary_key: the placeholder node of the graph to be fed
        # dictionary_value: the numpy array to feed to it
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_pred], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_pred.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_pred.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3.063594
Minibatch accuracy: 2.3%
Validation accuracy: 23.3%
Minibatch loss at step 500: 0.780721
Minibatch accuracy: 76.6%
Validation accuracy: 83.5%
Minibatch loss at step 1000: 0.447931
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 0.538234
Minibatch accuracy: 85.2%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 0.605932
Minibatch accuracy: 81.2%
Validation accuracy: 85.9%
Minibatch loss at step 2500: 0.514389
Minibatch accuracy: 86.7%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 0.470844
Minibatch accuracy: 85.2%
Validation accuracy: 86.9%
Minibatch loss at step 3500: 0.737191
Minibatch accuracy: 82.0%
Validation accuracy: 86.9%
Minibatch loss at step 4000: 0.653468
Minibatch accuracy: 82.8%
Validation accuracy: 86.9%
Minibatch loss at step 4500: 0.436758
Minibatch accuracy: 89.8%
Validation accuracy: 87.3%
Minibatch